In [65]:
# Names
# Will Kung, Ken Lu, Hunter Ross, Benjamin Sandoval

# Rain Prediction Model
We will build a model to predict whether it will rain tomorrow based on the historical data.

### Steps
1. Data Exploration/Cleaning

### Step 1: Data Exploration/Cleaning

We notice there are 343,248 NA cells (Missing values) in multiple colums across the data set. Here, we will:


In [66]:
# headers
import pandas as pd

In [67]:
# Load the CSV file
file_path = 'weatherAUS.csv'
original_df = pd.read_csv(file_path)

# Count the number of rows
original_df.shape[0]

145460

#### Before we do anything, it'd be a good idea to take a look at our dataset. (We printed the head here, but we also used Excel to get a better look at everything)

In [68]:
original_df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


#### We notice a couple things here. For one, missing data is marked with "NaN" (Or "NA" in the Excel file), which Pandas can detect as a missing value. So we don't have to do extra work there.

#### Secondly, there are many instances where the same attribute is missing across all records from a certain location. We assume this is due to that weather station just not measuring this value. This means we can't simply delete rows with missing values, because that would mean deleting all data from a location.

#### From here, we should get a better idea of our missing values and how many we're dealing with.

In [69]:
# Total number of NA values in each columns
print("All NA values:")
print(original_df.isna().sum())

All NA values:
Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


#### The values for Evaporation, Sunshine, Cloud9am, and Cloud39m stand out for how large they are (for reference, our dataset has about 145,460 records). We'll want to deal with those separately. For now, the missing values for RainToday and RainTomorrow draw our attention

#### There are some columns that just don't have a value for RainTomorrow. These datapoints won't be helpful in creating a prediction model, so we can take those off.RainToday is very tricky to replace, since we'd essentially have to have a whole other prediction model to say whether or not it would have rained that day, so we take out values that don't have that data as well (Note: It seems like the RainToday column was, for the most part, based off of the previous day's RainTomorrow. So not knowing a RainTomorrow value would make it very difficult to predict a RainToday).

In [70]:
# QUESTION: WE CAN PROBABLY TAKE THIS OUT, RIGHT?
# Number of NA rows in RainTomorrow & RainToday
na_counts = original_df[['RainTomorrow', 'RainToday']].isna().sum()
print("\nNA values in RainTomorrow and RainToday:")
print(na_counts)

# Drop NA rows in RainTomorrow & RainToday
# Note: New dataframe is created here. We want to keep original copy in case we want to use it in the future
df = original_df.dropna(subset=['RainTomorrow', 'RainToday'])
print(df.shape[0])


NA values in RainTomorrow and RainToday:
RainTomorrow    3267
RainToday       3261
dtype: int64
140787


### There is a class imbalance in RainTomorrow

In [71]:
rain_tomorrow_percentages = df['RainTomorrow'].value_counts(normalize=True) * 100

print(rain_tomorrow_percentages)

RainTomorrow
No     77.838153
Yes    22.161847
Name: proportion, dtype: float64


#### Some rows have more than half of its attributes with NaN values, we want to remove these rows because they will not be great for data prediction (We'd essentially be making up almost entire days)

In [72]:
# Drop rows with more than half of its columns with NA values
print(df.shape[0])
threshold = df.shape[1] // 2 # calculate half of the total columns
df = df.dropna(thresh=threshold)
print(df.shape[0])

140787
140553


##### TODO: DECIDE WHAT COLUMNS TO TAKE OUT BECAUSE THEY'RE TOO EMPTY

### Impute missing values based on location, group by RainTomorrow, and then group by location, calculate the mean or take a vote (categorical)

In [ ]:
for column in df.columns:
    # Impute values based on location
    if df[column].isnull().any():
        if df[column].dtype in ['float64', 'int64']:  # For numerical columns
            df[column] = df[column].fillna(
                df.groupby(['RainTomorrow', 'Location'])[column].transform('mean') #Do we have to use .transform()?
            )
        else:  # For categorical columns
            df[column] = df[column].fillna(
                df.groupby(['RainTomorrow', 'Location'])[column].transform(
                    lambda x: x.mode().iloc[0] if not x.mode().empty else None
                )
            )

    # Impute values based on date
    if df[column].isnull().any():
        if df[column].dtype in ['float64', 'int64']:  # For numerical columns
            df[column] = df[column].fillna(
                df.groupby(['RainTomorrow', 'Date'])[column].transform('mean')
            )
        else:  # For categorical columns
            df[column] = df[column].fillna(
                df.groupby(['RainTomorrow', 'Date'])[column].transform(
                    lambda x: x.mode().iloc[0] if not x.mode().empty else None
                )
            )

    # If there are still NA values, impute by column mean
    if df[column].isnull().any():
        if df[column].dtype in ['float64', 'int64']:  # For numerical columns
            df[column] = df[column].fillna( df.groupby(['RainTomorrow'])[column].transform('mean') )
        else:  # For categorical columns
            df[column] = df[column].fillna(
                df.groupby(['RainTomorrow'])[column].transform( lambda x: x.mode().iloc[0] if not x.mode().empty else None )
            )

print(df.isna().sum())
df.head()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,6.820000,11.211111,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.000000,4.80916,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,7.923077,9.954545,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,6.086585,4.80916,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,6.966667,10.520000,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,6.086585,2.00000,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,8.560000,7.400000,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,6.086585,4.80916,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,5.581818,8.222222,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No,No
